# sentiment analysis with textblob at the movie's caption. use time interval 1 sec, find positive, negative, neutral sentiments for each video. Have a dictionary with: the title of caption, the intervals  , the sentiments

# Imports

In [320]:
import pysrt
from datetime import date, datetime, timedelta, time
import pysrt
from textblob import TextBlob
import matplotlib
from matplotlib import style
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (16.0, 9.0)
style.use('fivethirtyeight')
import os

import csv
import os
import os, re, sys
from stat import *

import numpy as np
import datetime

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import csv

In [321]:
nltk.downloader.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/mscuser/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

# Handling srt Subtitles

# textblob sentiment Analysis

In [322]:
def textblob_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    return polarity
    

# Vader sentiment Analysis

In [288]:
'''VADER produces four sentiment metrics from these word ratings, 
which you can see below. The first three, positive, neutral and negative, 
represent the proportion of the text that falls into those categories. 
As you can see, our example sentence was rated as 45% positive, 
55% neutral and 0% negative. The final metric, the compound score, 
is the sum of all of the lexicon ratings (1.9 and 1.8 in this case) 
which have been standardised to range between -1 and 1. 
In this case, our example sentence has a rating of 0.69, which is pretty strongly positive.'''




sid = SentimentIntensityAnalyzer()
def vader_sentiment(text):
    polarity=sid.polarity_scores(text)['compound']
    return polarity



In [327]:

def get_sentiment(file):
    # Reading Subtitleget_sentiment
    subs = pysrt.open(file, encoding='iso-8859-1')
    n = len(subs)
    # List to store the time periods
    intervals = []
    sentiments_text_blob = []
    sentiments_vader = []

    # Collect and combine all the text in each time interval
    for i in range(n):
        text = ""
     
        # Finding all subtitle text in the each time interval
        segment=subs[i].end-subs[i].start
       # print "end:" + str(subs[i].end)+ "strat:" +str(subs[i].start) + "segment:"+str(segment) + "segment.to_time:"+str(segment.to_time())
        intervals.append(subs[i].start + segment)
        #for example [0,4],[4,6]->4,6
        if subs[i].end.to_time() <= (subs[i].start+ intervals[i]).to_time():
             text += subs[i].text_without_tags + " "
             
        else:
            break
        # Sentiment Analysis with TextBlob
  
        sentiment_blob=textblob_sentiment(text)
        sentiments_text_blob.append(sentiment_blob)
      
        # Sentiment Analysis with Vader
        sentiment_vader=vader_sentiment(text)
        sentiments_vader.append(sentiment_vader)
        
   # print sentiments_vader
   # print sentiments_text_blob
    #find the avrage of the 2 different sentiment analysis
    avg_sentiments=[(a_i + b_i)/float(2) for a_i, b_i in zip(sentiments_vader, sentiments_text_blob)]
  #  print avg_sentiments
    return (intervals, sentiments_text_blob)


In [328]:
# Utility to find average sentiment
def average(y):
    avg = float(sum(y))/len(y)
    return avg

In [329]:
#dict={id, path, , intervals, sentiments}
dict={}

i=interval_segments=sentiment_segments=0
def walktree(TopMostPath, callback):

    '''recursively descend the directory tree rooted at TopMostPath,
       calling the callback function for each regular file'''
    global dict
    for f in os.listdir(TopMostPath):
        pathname = os.path.join(TopMostPath, f)
        mode = os.stat(pathname)[ST_MODE]
        if S_ISDIR(mode):
            # It's a directory, recurse into it
            walktree(pathname, callback)
        elif S_ISREG(mode):
            # It's a file, call the callback function
            callback(pathname)
        else:
            # Unknown file type, print a message
            print 'Skipping %s' % pathname


def sentiment(file):
    
    global i 
    global interval_segments, sentiment_segments
    #the srt files we need to train our model
    if '.srt' in file:
        interval_segments, sentiment_segments = get_sentiment(file)
        #fig, ax = plt.subplots()
        #plt.plot(interval_segments,sentiment_segments)
        #plt.title(file, fontsize=32)
        #plt.ylim((-1, 1))
        #plt.ylabel("Sentiment Polarity")
        #plt.xlabel("Running Time")
        #plt.text(.5, 1.03, "Average Sentiment - " + str(round(average(y), 4)), color="green")
        
        #DICTIONARY PATH,INTERVALS,SENTIMENTS
        dict[i]={}
        dict[i]['Id']=i
        dict[i]['path']=file
        dict[i]['intervals']=interval_segments
        dict[i]['sentiments']=sentiment_segments
        i=i+1
        
        


In [330]:
path = '/home/mscuser/multimodal/multimodal_audio'
walktree(path, sentiment)


# Make a csv for all videos that contains:videoid, path, list of interval segments, list of sentiments

In [331]:
for i in range(len(dict)):
    with open('captions_polarity.csv', 'wb') as f:
        writer = csv.writer(f)
        for key, value in dict.items():
            writer.writerow([key, value])
            
            

In [332]:

#print len(sentiment_segments)
#print sentiment_segments

In [333]:
#print interval_segments[0]
#print interval_segments[1]
#print interval_segments[99]

# Make a csv for each video that contains: segment interval, pollarity for all  segments of the video.  The name of the file is polarity follwed by videoId. eg:polarity0

In [334]:
for i in range(len(dict)):
    new_csv='polarity'+str(dict[i]['Id']) +'.csv'
    with open(new_csv, 'wb') as f:
        writer = csv.writer(f)
        for j in range(len(dict[i]['intervals'])):
            context=(dict[i]['intervals'][j],dict[i]['sentiments'][j])
            writer.writerows([context])
            
            
            
            
            
          

polarity0.csv
